In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop

/Users/tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Agent

In [2]:
class DDQN_Agent:
    #
    # Initializes attributes and constructs CNN model and target_model
    #
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        
        # Hyperparameters
        self.gamma = 0.99           # Discount rate
        self.epsilon = 1.0          # Exploration rate
        self.epsilon_min = 0.1      # Minimal exploration rate (epsilon-greedy)
        self.epsilon_decay = 0.995  # Decay rate for epsilon
        self.update_rate = 10000    # Number of steps until updating the target network
        
        # Construct DQN models
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary()

    #
    # Constructs CNN
    #
    def _build_model(self):
        model = Sequential()
        
        # Conv Layers
        model.add(Conv2D(32, (8, 8), strides=4, padding='same', input_shape=self.state_size))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())

        # FC Layers
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=RMSprop(lr=0.00025, rho=0.95, epsilon=None, decay=0.0)
)
        return model

    #
    # Stores experience in replay memory
    #
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    #
    # Chooses action based on epsilon-greedy policy
    #
    def act(self, state):
        # Random exploration
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        
        return np.argmax(act_values[0])  # Returns action using policy

    #
    # Trains the model using randomly selected experiences in the replay memory
    #
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            
            if not done:
                max_action = np.argmax(self.model.predict(next_state)[0])
                target = (reward + self.gamma * self.target_model.predict(next_state)[0][max_action])
            else:
                target = reward
                
            # Construct the target vector as follows:
            # 1. Use the current model to output the Q-value predictions
            target_f = self.model.predict(state)
            
            # 2. Rewrite the chosen action value with the computed target
            target_f[0][action] = target
            
            # 3. Use vectors in the objective computation
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    #
    # Sets the target model parameters to the current model parameters
    #
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
            
    #
    # Loads a saved model
    #
    def load(self, name):
        self.model.load_weights(name)

    #
    # Saves parameters of a trained model
    #
    def save(self, name):
        self.model.save_weights(name)

# Preprocessing

In [3]:
# Helpful preprocessing taken from github.com/ageron/tiny-dqn
def process_frame(frame):

    mspacman_color = np.array([210, 164, 74]).mean()
    img = frame[1:176:2, ::2]    # Crop and downsize
    img = img.mean(axis=2)       # Convert to greyscale
    img[img==mspacman_color] = 0 # Improve contrast by making pacman white
    img = (img - 128) / 128 - 1  # Normalize from -1 to 1.
    
    return np.expand_dims(img.reshape(88, 80, 1), axis=0)

In [4]:
# Averages images from the last few frame
def  blend_images (images, blend):
    avg_image = np.expand_dims(np.zeros((88, 80, 1), np.float64), axis=0)

    for image in images:
        avg_image += image
        
    if len(images) < blend:
        return avg_image / len(images)
    else:
        return avg_image / blend

# Environment

In [5]:
env = gym.make('MsPacman-v0')
state_size = (88, 80, 1)
action_size = env.action_space.n
agent = DDQN_Agent(state_size, action_size)
#agent.load('models/')

episodes = 5000
batch_size = 32
skip_start = 90  # MsPacman-v0 waits for 90 actions before the episode begins
total_time = 0   # Counter for total number of steps taken
all_rewards = 0  # Used to compute avg reward over time
blend = 4        # Number of images to blend
done = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 20, 32)        2080      
_________________________________________________________________
activation_1 (Activation)    (None, 22, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 10, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 10, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7040)              0         
__________

In [ ]:
for e in range(episodes):
    total_reward = 0
    game_score = 0
    state = process_frame(env.reset())
    images = deque(maxlen=blend)  # Array of images to be blended
    images.append(state)
    
    for skip in range(skip_start): # skip the start of each game
        env.step(0)
    
    for time in range(20000):
        env.render()
        total_time += 1
        
        # Every update_rate timesteps we update the target network parameters
        if total_time % agent.update_rate == 0:
            agent.update_target_model()
        
        # Return the avg of the last 4 frames
        state = blend_images(images, blend)
        
        # Transition Dynamics
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        
        game_score += reward
        total_reward += reward
        
        # Return the avg of the last 4 frames
        next_state = process_frame(next_state)
        images.append(next_state)
        next_state = blend_images(images, blend)
        
        # Store sequence in replay memory
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            all_rewards += game_score
            
            print("episode: {}/{}, game score: {}, reward: {}, avg reward: {}, time: {}, total time: {}"
                  .format(e+1, episodes, game_score, total_reward, all_rewards/(e+1), time, total_time))
            
            break
            
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 1/5000, game score: 240.0, reward: 240.0, avg reward: 240.0, time: 617, total time: 618
episode: 2/5000, game score: 360.0, reward: 360.0, avg reward: 300.0, time: 556, total time: 1175
episode: 3/5000, game score: 230.0, reward: 230.0, avg reward: 276.666666667, time: 659, total time: 1835
episode: 4/5000, game score: 280.0, reward: 280.0, avg reward: 277.5, time: 603, total time: 2439
episode: 5/5000, game score: 220.0, reward: 220.0, avg reward: 266.0, time: 621, total time: 3061
episode: 6/5000, game score: 280.0, reward: 280.0, avg reward: 268.333333333, time: 601, total time: 3663
episode: 7/5000, game score: 230.0, reward: 230.0, avg reward: 262.857142857, time: 626, total time: 4290
episode: 8/5000, game score: 290.0, reward: 290.0, avg reward: 266.25, time: 789, total time: 5080
episode: 9/5000, game score: 270.0, reward: 270.0, avg reward: 266.666666667, time: 543, total time: 5624
episode: 10/5000, game score: 230.0, reward: 230.0, avg reward: 263.0, time: 430, tota

episode: 79/5000, game score: 310.0, reward: 310.0, avg reward: 325.949367089, time: 624, total time: 49331
episode: 80/5000, game score: 300.0, reward: 300.0, avg reward: 325.625, time: 587, total time: 49919
episode: 81/5000, game score: 320.0, reward: 320.0, avg reward: 325.555555556, time: 540, total time: 50460
episode: 82/5000, game score: 250.0, reward: 250.0, avg reward: 324.634146341, time: 630, total time: 51091
episode: 83/5000, game score: 240.0, reward: 240.0, avg reward: 323.614457831, time: 568, total time: 51660
episode: 84/5000, game score: 230.0, reward: 230.0, avg reward: 322.5, time: 573, total time: 52234
episode: 85/5000, game score: 1020.0, reward: 1020.0, avg reward: 330.705882353, time: 713, total time: 52948
episode: 86/5000, game score: 260.0, reward: 260.0, avg reward: 329.88372093, time: 653, total time: 53602
episode: 87/5000, game score: 450.0, reward: 450.0, avg reward: 331.264367816, time: 922, total time: 54525
episode: 88/5000, game score: 310.0, rewa

episode: 156/5000, game score: 260.0, reward: 260.0, avg reward: 328.205128205, time: 503, total time: 100119
episode: 157/5000, game score: 240.0, reward: 240.0, avg reward: 327.643312102, time: 584, total time: 100704
episode: 158/5000, game score: 330.0, reward: 330.0, avg reward: 327.658227848, time: 595, total time: 101300
episode: 159/5000, game score: 340.0, reward: 340.0, avg reward: 327.735849057, time: 582, total time: 101883
episode: 160/5000, game score: 300.0, reward: 300.0, avg reward: 327.5625, time: 594, total time: 102478
episode: 161/5000, game score: 310.0, reward: 310.0, avg reward: 327.453416149, time: 774, total time: 103253
episode: 162/5000, game score: 660.0, reward: 660.0, avg reward: 329.50617284, time: 926, total time: 104180
episode: 163/5000, game score: 280.0, reward: 280.0, avg reward: 329.202453988, time: 545, total time: 104726
episode: 164/5000, game score: 380.0, reward: 380.0, avg reward: 329.512195122, time: 704, total time: 105431
episode: 165/500

episode: 231/5000, game score: 340.0, reward: 340.0, avg reward: 323.636363636, time: 775, total time: 145682
episode: 232/5000, game score: 140.0, reward: 140.0, avg reward: 322.844827586, time: 496, total time: 146179
episode: 233/5000, game score: 150.0, reward: 150.0, avg reward: 322.103004292, time: 575, total time: 146755
episode: 234/5000, game score: 130.0, reward: 130.0, avg reward: 321.282051282, time: 377, total time: 147133
episode: 235/5000, game score: 320.0, reward: 320.0, avg reward: 321.276595745, time: 606, total time: 147740
episode: 236/5000, game score: 280.0, reward: 280.0, avg reward: 321.101694915, time: 557, total time: 148298
episode: 237/5000, game score: 230.0, reward: 230.0, avg reward: 320.717299578, time: 673, total time: 148972
episode: 238/5000, game score: 280.0, reward: 280.0, avg reward: 320.546218487, time: 519, total time: 149492
episode: 239/5000, game score: 580.0, reward: 580.0, avg reward: 321.631799163, time: 777, total time: 150270
episode: 2

episode: 306/5000, game score: 240.0, reward: 240.0, avg reward: 305.849673203, time: 421, total time: 187207
episode: 307/5000, game score: 230.0, reward: 230.0, avg reward: 305.602605863, time: 506, total time: 187714
episode: 308/5000, game score: 160.0, reward: 160.0, avg reward: 305.12987013, time: 369, total time: 188084
episode: 309/5000, game score: 280.0, reward: 280.0, avg reward: 305.048543689, time: 603, total time: 188688
episode: 310/5000, game score: 260.0, reward: 260.0, avg reward: 304.903225806, time: 579, total time: 189268
episode: 311/5000, game score: 180.0, reward: 180.0, avg reward: 304.501607717, time: 522, total time: 189791
episode: 312/5000, game score: 180.0, reward: 180.0, avg reward: 304.102564103, time: 481, total time: 190273
episode: 313/5000, game score: 190.0, reward: 190.0, avg reward: 303.738019169, time: 607, total time: 190881
episode: 314/5000, game score: 220.0, reward: 220.0, avg reward: 303.47133758, time: 530, total time: 191412
episode: 315

episode: 381/5000, game score: 170.0, reward: 170.0, avg reward: 293.884514436, time: 421, total time: 230083
episode: 382/5000, game score: 230.0, reward: 230.0, avg reward: 293.717277487, time: 587, total time: 230671
episode: 383/5000, game score: 190.0, reward: 190.0, avg reward: 293.446475196, time: 458, total time: 231130
episode: 384/5000, game score: 220.0, reward: 220.0, avg reward: 293.255208333, time: 534, total time: 231665
episode: 385/5000, game score: 170.0, reward: 170.0, avg reward: 292.935064935, time: 556, total time: 232222
episode: 386/5000, game score: 160.0, reward: 160.0, avg reward: 292.590673575, time: 487, total time: 232710
episode: 387/5000, game score: 130.0, reward: 130.0, avg reward: 292.170542636, time: 389, total time: 233100
episode: 388/5000, game score: 130.0, reward: 130.0, avg reward: 291.75257732, time: 342, total time: 233443
episode: 389/5000, game score: 230.0, reward: 230.0, avg reward: 291.593830334, time: 616, total time: 234060
episode: 39

episode: 456/5000, game score: 150.0, reward: 150.0, avg reward: 281.535087719, time: 588, total time: 271268
episode: 457/5000, game score: 160.0, reward: 160.0, avg reward: 281.269146608, time: 482, total time: 271751
episode: 458/5000, game score: 470.0, reward: 470.0, avg reward: 281.681222707, time: 859, total time: 272611
episode: 459/5000, game score: 260.0, reward: 260.0, avg reward: 281.633986928, time: 599, total time: 273211
episode: 460/5000, game score: 180.0, reward: 180.0, avg reward: 281.413043478, time: 428, total time: 273640
episode: 461/5000, game score: 150.0, reward: 150.0, avg reward: 281.127982646, time: 547, total time: 274188
episode: 462/5000, game score: 260.0, reward: 260.0, avg reward: 281.082251082, time: 524, total time: 274713
episode: 463/5000, game score: 280.0, reward: 280.0, avg reward: 281.079913607, time: 546, total time: 275260
episode: 464/5000, game score: 180.0, reward: 180.0, avg reward: 280.862068966, time: 516, total time: 275777
episode: 4

In [12]:
agent.save('models/breakout_ddqn_50k-memory_783-steps')